<a href="https://colab.research.google.com/github/arshad-nx/Projects/blob/main/Pandas_Introduction_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Introduction (Excel → DataFrame → GroupBy → Merge)

A beginner-friendly notebook for **Pandas**. We move from Excel thinking (tables, filters, pivots) to pandas workflows.

**You will learn**
- Reading CSVs (including Colab upload)
- DataFrame basics: columns, rows, head, shape, info
- Selecting columns/rows and filtering
- Sorting and creating new columns
- GroupBy (Excel PivotTable logic)
- Missing values basics
- Merging two tables (like VLOOKUP / Power Pivot relationships)
- Exporting cleaned data


---
## 0) Setup

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

print("✅ pandas version:", pd.__version__)

---
## 1) Create a DataFrame (Excel table in Python)
We start with a small table to understand the basics.

In [ ]:
data = {
    "Customer": ["Asha", "Rohit", "Meera", "Kabir", "Zoya"],
    "City": ["Pune", "Mumbai", "Pune", "Delhi", "Mumbai"],
    "Orders": [3, 5, 2, 6, 1],
    "Spend": [1200, 2500, 800, 3100, 450]
}

df = pd.DataFrame(data)
df

### 1.1 Quick checks (like looking at an Excel sheet)
- `head()` preview
- `shape` rows/cols
- `columns` list of columns
- `info()` summary

In [ ]:
display(df.head())
print("shape:", df.shape)
print("columns:", list(df.columns))
df.info()

---
## 2) Selecting columns and rows
Excel analogy: selecting a column, filtering rows.

In [ ]:
# Single column -> Series
print(df["Spend"])

# Multiple columns -> DataFrame
display(df[["Customer", "City", "Spend"]])

In [ ]:
# Row selection by position with iloc
display(df.iloc[0:3])

# Row selection by condition (filter)
display(df[df["City"] == "Pune"])

---
## 3) Filtering with multiple conditions (AND/OR)
Excel analogy: multiple filters at once.

In [ ]:
# Customers in Mumbai with Spend > 1000
filtered = df[(df["City"] == "Mumbai") & (df["Spend"] > 1000)]
display(filtered)

# Customers in Pune OR Orders >= 5
filtered2 = df[(df["City"] == "Pune") | (df["Orders"] >= 5)]
display(filtered2)

---
## 4) Sorting and creating new columns
Very common in data cleaning.

In [ ]:
# Sort by Spend descending
sorted_df = df.sort_values("Spend", ascending=False)
display(sorted_df)

In [ ]:
# Create a new column: average spend per order
# (Spend / Orders)
df["Avg_Spend_Per_Order"] = df["Spend"] / df["Orders"]
display(df)

In [ ]:
# Create a category column (like IF in Excel)
df["Spend_Level"] = np.where(df["Spend"] >= 2000, "High", "Low")
display(df)

---
## 5) GroupBy (PivotTable logic)
Group by City and compute summary stats.

In [ ]:
city_summary = df.groupby("City", as_index=False).agg(
    Customers_Count=("Customer", "count"),
    Total_Spend=("Spend", "sum"),
    Avg_Spend=("Spend", "mean"),
    Total_Orders=("Orders", "sum")
)
city_summary

---
## 6) A larger realistic dataset (synthetic)
We generate a small retail-style dataset for practice.

In [ ]:
np.random.seed(11)

n = 200
cities = np.random.choice(["Pune", "Mumbai", "Delhi", "Bengaluru"], size=n)
segments = np.random.choice(["Budget", "Regular", "Premium"], size=n, p=[0.35, 0.45, 0.20])
orders = np.random.randint(1, 9, size=n)
base_spend = np.where(segments == "Budget", 500, np.where(segments == "Regular", 1200, 2200))
spend = base_spend + orders*120 + np.random.normal(0, 200, size=n)

retail = pd.DataFrame({
    "City": cities,
    "Segment": segments,
    "Orders": orders,
    "Spend": np.round(spend, 0).astype(int)
})

retail.head()

### 6.1 Quick EDA checks

In [ ]:
print("shape:", retail.shape)
print("missing values:\n", retail.isna().sum())
display(retail.describe(numeric_only=True))

---
## 7) Missing values (simple introduction)
We will purposely create a few missing values and handle them.

In [ ]:
retail_missing = retail.copy()

# Introduce missing values in Spend for a few rows
idx = np.random.choice(retail_missing.index, size=8, replace=False)
retail_missing.loc[idx, "Spend"] = np.nan

print("Missing values:\n", retail_missing.isna().sum())
retail_missing.head()

In [ ]:
# Option A: Drop rows with missing values (simple)
dropped = retail_missing.dropna()
print("Original rows:", len(retail_missing), "After dropna:", len(dropped))

In [ ]:
# Option B: Fill missing Spend with median Spend (common)
filled = retail_missing.copy()
filled["Spend"] = filled["Spend"].fillna(filled["Spend"].median())
print("Missing after fill:", filled["Spend"].isna().sum())
filled.head()

---
## 8) Merge two tables (VLOOKUP / Power Pivot style)
We’ll create a Customer table and an Orders table and merge them.

In [ ]:
customers = pd.DataFrame({
    "Customer_ID": [101, 102, 103, 104, 105],
    "Customer_Name": ["Asha", "Rohit", "Meera", "Kabir", "Zoya"],
    "City": ["Pune", "Mumbai", "Pune", "Delhi", "Mumbai"]
})

orders = pd.DataFrame({
    "Order_ID": [1, 2, 3, 4, 5, 6, 7],
    "Customer_ID": [101, 101, 102, 103, 103, 104, 105],
    "Order_Value": [500, 700, 1200, 400, 650, 1400, 300]
})

display(customers)
display(orders)

In [ ]:
# Merge (like bringing customer fields into the orders table)
merged = orders.merge(customers, on="Customer_ID", how="left")
merged

In [ ]:
# PivotTable-like summary: total order value by City
city_orders = merged.groupby("City", as_index=False)["Order_Value"].sum().sort_values("Order_Value", ascending=False)
city_orders

---
## 9) CSV Upload in Colab (real workflow)
Use this when students have a dataset like `advertising.csv`.

In [ ]:
from google.colab import files
uploaded = files.upload()
print("Uploaded files:", list(uploaded.keys()))

### 9.1 Read the uploaded file
Update `FILENAME` to match your uploaded file name.

In [ ]:
FILENAME = "your_file.csv"  # ✅ CHANGE THIS

try:
    user_df = pd.read_csv(FILENAME)
    display(user_df.head())
    print("shape:", user_df.shape)
    print("columns:", list(user_df.columns))
except FileNotFoundError:
    print("❌ File not found. Check filename in the left Files panel and update FILENAME.")

---
## 10) Export cleaned data
After cleaning/filtering, export back to CSV.

In [ ]:
# Example: export retail summary
summary = retail.groupby(["City", "Segment"], as_index=False).agg(
    Avg_Spend=("Spend", "mean"),
    Total_Spend=("Spend", "sum"),
    Total_Orders=("Orders", "sum")
)

out = "retail_summary.csv"
summary.to_csv(out, index=False)
print("✅ Saved:", out)
summary.head()

---
## 11) Exercises (copy-paste friendly)
### Easy
1. In `retail`, filter only `City == 'Pune'` and show the first 10 rows.
2. Sort `retail` by `Spend` descending.
3. Create a new column `Spend_per_Order = Spend / Orders`.

### Medium
4. Create a PivotTable-like summary: `Avg Spend` and `Total Spend` by `Segment`.
5. Create a filter: Premium customers in Mumbai with Spend > 2500.

### Challenge
6. Merge two tables of your choice (example: products + sales) and compute total sales by product category.


In [ ]:
# Write your solutions here

# Example starter:
# pune = retail[retail['City'] == 'Pune']
# display(pune.head(10))
